In [1]:
markup_folder_path = "/home/devel/NVI/outputs/my_notebook/markup"

In [4]:
!sudo pip install pytorch_lightning ipdb ipymarkup razdel
!sudo pip install torchmetrics==0.10.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.8/797.8 kB 20.7 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 28.3 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=cba7aea4d1e9478fefa5af2cd6af44842b8fc616f99731a0812661fa3a9b1cfe
  Stored in directory: /root/.cache/pip/wheels/45/23/de/5789a92962483fd33cb06674792b9697c1b3766d7c7742830e
Successfully built intervaltree
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━

In [5]:
!sudo pip install ipymarkup
!sudo pip install ipywidgets

  Using cached ipymarkup-0.9.0-py3-none-any.whl (14 kB)
  Using cached intervaltree-3.1.0-py2.py3-none-any.whl

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 37.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import re
from xml.etree import ElementTree
import os
import random
from tqdm import tqdm
from collections import Counter, namedtuple
from sklearn.metrics import confusion_matrix
from enum import IntEnum
from razdel import tokenize
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN, PURPLE, BROWN

import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter, namedtuple
from enum import IntEnum



import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler
from torchmetrics.classification.accuracy import Accuracy


import torch
import torch.nn as nn
import torchmetrics
from pytorch_lightning import LightningModule
from torchmetrics import Accuracy
import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from collections import defaultdict

In [3]:
def get_property_annotation(start, end, child, node_id):
  dct = defaultdict(list)
  for annotations in child:
    if 'annotation' in annotations.tag and "PropertyAnnotation" in annotations[0].tag:
      property_node_id = [i for i in annotations[0][2].attrib.values()][0]
      if property_node_id != node_id:
        continue
      property_start = int(annotations[0][0].text)
      property_end = int(annotations[0][1].text)
      if start <= property_start and property_end <= end:
        key = re.findall(r"#\w*", annotations[0][3].text)[0][1:]
        dct[key].append([property_start - start, property_end - start])
  return dct

In [4]:
def get_annotation_dict(file_name):
  tree = ElementTree.parse(os.path.join(markup_folder_path, file_name)) 
  root = tree.getroot()

  annotation = {}
  for child in root:
    if "TextAnnotations" in child.tag:
      for text_anotations in child:
        if 'document_text' in text_anotations.tag:
          text = text_anotations.text
        else:
          if "InstanceAnnotation" in text_anotations[0].tag:
            start = int(text_anotations[0][0].text)
            end = int(text_anotations[0][1].text)
            node_id = [i for i in text_anotations[0][2].attrib.values()][0]
            dct = get_property_annotation(start, end, child, node_id)
            if len(dct) == 0:
                continue
            annotation[text[start : end]] = dct
  return annotation

In [5]:
get_annotation_dict("3373424.xml")

{'one million Omani Rials': defaultdict(list,
             {'ms_major_amount': [[0, 11]], 'ms_major_currency': [[12, 23]]}),
 'RO300\xa0million': defaultdict(list,
             {'ms_major_currency': [[0, 2]], 'ms_major_amount': [[2, 13]]}),
 'RO 1826.4\xa0million': defaultdict(list,
             {'ms_major_currency': [[0, 2]], 'ms_major_amount': [[3, 17]]}),
 'RO50\xa0million': defaultdict(list,
             {'ms_major_currency': [[0, 2]], 'ms_major_amount': [[2, 12]]}),
 'RO10\xa0million': defaultdict(list,
             {'ms_major_currency': [[0, 2]], 'ms_major_amount': [[2, 12]]})}

In [71]:
data = []
for file in tqdm(os.listdir(markup_folder_path)):
  if '.xml' not in file:
    continue
  annotation = get_annotation_dict(file)
  for key, value in annotation.items():
    lst = []
    for cls, intervals in value.items():
      for interval in intervals:
        lst.append((interval[0], interval[1], cls))
    data.append((key, lst))

100% 6264/6264 [00:01<00:00, 4858.89it/s]


In [87]:
def my_tokenize(text):
    tokenized = list(tokenize(text))
    pattern = r"([-_MT€$₽£¥₣₹￠₫])"
    new_tokens = []
    for token in tokenized:
        annotations = list(filter(None, re.split(pattern, token.text)))
        start = token.start
        for annot in annotations:
            token_annot = list(tokenize(annot))
            token_annot[0].start = start
            token_annot[0].stop = start + len(annot)
            start += len(annot)
            new_tokens.append(token_annot[0])
    # merge same near symbols
    i = 0
    while(True):
        if i >= len(new_tokens) - 1:
            break
        if new_tokens[i].text[-1] == new_tokens[i+1].text[0] and new_tokens[i].stop == new_tokens[i+1].start:
            new_tokens[i].text += new_tokens[i+1].text
            new_tokens[i].stop = new_tokens[i+1].stop
            new_tokens.pop(i+1)
            i -= 1
        i += 1
    return new_tokens

In [88]:
my_tokenize("₹one")

[Substring(0, 1, '₹'), Substring(1, 4, 'one')]

In [186]:
class Label(IntEnum):
    MAJOR_AMOUNT = 1
    MAJOR_CURRENCY = 2
    MINOR_AMOUNT = 3
    MINOR_CURRENCY = 4
    AMOUNT_BRACKETED = 5
    LIKE_BRACKETS = 6

Sample = namedtuple("Sample","text,tokens,spans,labels")

def text_span_to_sample(text, spans):
  labels = []
  tokens = my_tokenize(text)
  for token in tokens:
        label = Label.LIKE_BRACKETS
        for span in spans:
            span_begin, span_end, tag = span
            if token.start >= span_begin and token.stop <= span_end:
                if tag == 'ms_major_amount':
                  label = Label.MAJOR_AMOUNT
                elif tag == 'ms_major_currency':
                  label = Label.MAJOR_CURRENCY
                elif tag == 'ms_minor_amount':
                  label = Label.MINOR_AMOUNT
                elif tag == 'ms_minor_currency':
                  label = Label.MINOR_CURRENCY
                elif tag == 'ms_major_amount_bracketed':
                  label = Label.AMOUNT_BRACKETED          
        labels.append(label)
  return Sample(text, tokens, spans, labels)

samples = []
for text, spans in data:
    samples.append(text_span_to_sample(text, spans))

show_box_markup(samples[8683].text, samples[8683].spans, palette=palette(ms_major_currency=BLUE, ms_major_amount=RED, ms_minor_amount=GREEN, ms_minor_currency=PURPLE, ms_major_amount_bracketed=BROWN))
print(samples[8683].labels)
print(len(samples))

[<Label.MAJOR_CURRENCY: 2>, <Label.MAJOR_AMOUNT: 1>]
17210


In [225]:
random.shuffle(samples)

train = samples[:11500]
val = samples[11500:13500]
test = samples[13500:]

In [226]:
import csv

with open('input/train1.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for index in range(len(train)):
        for token_index in range(len(train[index].tokens)):
            tsv_writer.writerow([train[index].tokens[token_index].text, train[index].labels[token_index].name])
        tsv_writer.writerow(['', ''])

with open('input/test1.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for index in range(len(test)):
        for token_index in range(len(test[index].tokens)):
            tsv_writer.writerow([test[index].tokens[token_index].text, test[index].labels[token_index].name])
        tsv_writer.writerow(['', ''])

with open('input/val1.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for index in range(len(val)):
        for token_index in range(len(val[index].tokens)):
            tsv_writer.writerow([val[index].tokens[token_index].text, val[index].labels[token_index].name])
        tsv_writer.writerow(['', ''])

In [227]:
char_set = ["<pad>", "<unk>"] + list({ch for sample in train for token in sample.tokens for ch in token.text})
print(char_set)
print(len(char_set))

['<pad>', '<unk>', '1', '₤', 'i', 'd', 'ő', 'F', '/', 'ą', 'м', 'م', 'j', 'o', '2', '￦', '₡', '7', 'ê', '5', '£', 'ó', 'á', '¢', '¼', 'Z', 'z', 'e', 'п', ',', '₪', 'h', '9', 'u', 'f', '₧', '¾', '￡', 'E', 'N', 'w', 'm', '"', 'v', '₭', 't', '₵', 'J', 'y', 'W', 'X', 'ì', 'ł', 'g', 'ƒ', '⁄', 'U', 'đ', '₽', 'ر', '฿', '_', '₹', '₱', 'ع', 'c', 'a', 'l', '¥', 'V', '₺', 'ā', ']', 'ø', '$', '₾', 'ạ', 'с', 'L', 'n', '₯', 'S', '4', 'к', '€', 'q', 'ý', 'ä', '(', '½', '6', '0', '”', 'C', 'A', '·', '₫', 'í', '\u200b', 'B', 'k', ')', 'O', '.', 'ồ', '₼', 'ö', 'é', 'ề', '₮', 'p', '“', '’', '+', "'", 'ل', 'x', '₴', 'R', '-', '[', '3', '￥', 'Y', 'P', 'о', '៛', 'b', '⅛', '8', 'M', 's', 'Q', 'I', 'K', 'r', 'H', '₦', 'D', ':', 'T', 'G', 'د', '৳', '₩', '₨']
146


In [228]:
max_seq_len = 0
max_char_seq_len = 0

for sample in samples:
  max_seq_len = max(len(sample.tokens), max_seq_len)
  if len(sample.tokens) == 27:
    print(sample.text)
    print()
  for token in sample.tokens:
    max_char_seq_len = max(len(token.text), max_char_seq_len)
    if len(token.text) == 29:
        print(token.text)
        print(sample.text)

Sixty-Three Billion Five Hundred Sixty Million Eight Hundred Thirty-Seven Thousand Two Hundred Sixty-Nine South Korean Won

1,200,000,000,000,000,000,000
1.2 sextillion (1,200,000,000,000,000,000,000)


In [229]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [15]:
!pip install torchtext==0.6.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [18]:
!pip install pytorch-crf

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [230]:
import time
#import gensim
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.optim import Adam
from torchtext.data import Field, NestedField, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from torchtext.vocab import Vocab
from torchcrf import CRF
from collections import Counter

In [231]:
class Corpus(object):

    def __init__(self, input_folder, min_word_freq, batch_size, wv_file=None):
        # list all the fields
        self.word_field = Field(lower=True)  # [sent len, batch_size]
        self.tag_field = Field(unk_token=None)  # [sent len, batch_size]
        # Character-level input
        self.char_nesting_field = Field(tokenize=list)
        self.char_field = NestedField(self.char_nesting_field)  # [batch_size, sent len, max len char]
        # create dataset using built-in parser from torchtext
        self.train_dataset, self.val_dataset, self.test_dataset = SequenceTaggingDataset.splits(
            path=input_folder,
            train="train1.tsv",
            validation="val1.tsv",
            test="test1.tsv",
            fields=(
                (("word", "char"), (self.word_field, self.char_field)),
                ("tag", self.tag_field)
            )
        )
        # convert fields to vocabulary list
        self.word_field.build_vocab(self.train_dataset.word, min_freq=min_word_freq)
        # build vocab for tag and characters
        self.char_field.build_vocab(self.train_dataset.char)
        self.tag_field.build_vocab(self.train_dataset.tag)
        # create iterator for batch input
        self.train_iter, self.val_iter, self.test_iter = BucketIterator.splits(
            datasets=(self.train_dataset, self.val_dataset, self.test_dataset),
            batch_size=batch_size
        )
        # prepare padding index to be ignored during model training/evaluation
        self.word_pad_idx = self.word_field.vocab.stoi[self.word_field.pad_token]
        self.char_pad_idx = self.char_field.vocab.stoi[self.char_field.pad_token]
        self.tag_pad_idx = self.tag_field.vocab.stoi[self.tag_field.pad_token]

In [232]:
corpus = Corpus('/home/devel/NVI/outputs/my_notebook/input/', 1, batch_size = 16)

In [233]:
corpus.train_dataset[0].__dict__

{'word': ['pound', 'sterling', '185,563.50'],
 'char': [['p', 'o', 'u', 'n', 'd'],
  ['s', 't', 'e', 'r', 'l', 'i', 'n', 'g'],
  ['1', '8', '5', ',', '5', '6', '3', '.', '5', '0']],
 'tag': ['MAJOR_CURRENCY', 'MAJOR_CURRENCY', 'MAJOR_AMOUNT']}

In [234]:
class BiLSTM(nn.Module):

    def __init__(self,
                 input_dim,
                 embedding_dim,
                 char_emb_dim,
                 char_input_dim,
                 char_cnn_filter_num,
                 char_cnn_kernel_size,
                 hidden_dim,
                 output_dim,
                 lstm_layers,
                 attn_heads,
                 emb_dropout,
                 cnn_dropout,
                 lstm_dropout,
                 attn_dropout,
                 fc_dropout,
                 word_pad_idx,  # NEWLY ADDED
                 char_pad_idx,
                 tag_pad_idx):
        super().__init__()
        self.char_pad_idx = char_pad_idx  # NEWLY ADDED
        self.word_pad_idx = word_pad_idx  # NEWLY ADDED
        self.tag_pad_idx = tag_pad_idx  # NEWLY ADDED
        # LAYER 1A: Word Embedding
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(
            num_embeddings=input_dim,
            embedding_dim=embedding_dim,
            padding_idx=word_pad_idx
        )
        self.emb_dropout = nn.Dropout(emb_dropout)
        # LAYER 1B: Char Embedding-CNN
        self.char_emb_dim = char_emb_dim
        self.char_emb = nn.Embedding(
            num_embeddings=char_input_dim,
            embedding_dim=char_emb_dim,
            padding_idx=char_pad_idx
        )
        self.char_cnn = nn.Conv1d(
            in_channels=char_emb_dim,
            out_channels=char_emb_dim * char_cnn_filter_num,
            kernel_size=char_cnn_kernel_size,
            groups=char_emb_dim  # different 1d conv for each embedding dim
        )
        self.cnn_dropout = nn.Dropout(cnn_dropout)
        # LAYER 2: BiLSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim + (char_emb_dim * char_cnn_filter_num),
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            bidirectional=True,
            dropout=lstm_dropout if lstm_layers > 1 else 0
        )
        # LAYER 3: Self-attention
        self.attn = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,
            num_heads=attn_heads,
            dropout=attn_dropout
        )
        # LAYER 4: Fully-connected
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # times 2 for bidirectional
        # LAYER 5: CRF
        self.crf = CRF(num_tags=output_dim)
        # init weights from normal distribution
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1)

    def forward(self, words, chars, tags=None):
        # words = [sentence length, batch size]
        # chars = [batch size, sentence length, word length)
        # tags = [sentence length, batch size]
        # embedding_out = [sentence length, batch size, embedding dim]
        embedding_out = self.emb_dropout(self.embedding(words))
        # character cnn layer forward
        # reference: https://github.com/achernodub/targer/blob/master/src/layers/layer_char_cnn.py
        # char_emb_out = [batch size, sentence length, word length, char emb dim]
        char_emb_out = self.emb_dropout(self.char_emb(chars))
        batch_size, sent_len, word_len, char_emb_dim = char_emb_out.shape
        char_cnn_max_out = torch.zeros(batch_size, sent_len, self.char_cnn.out_channels)
        for sent_i in range(sent_len):
            # sent_char_emb = [batch size, word length, char emb dim]
            sent_char_emb = char_emb_out[:, sent_i, :, :]
            # sent_char_emb_p = [batch size, char emb dim, word length]
            sent_char_emb_p = sent_char_emb.permute(0, 2, 1)
            # char_cnn_sent_out = [batch size, out channels * char emb dim, word length - kernel size + 1]
            char_cnn_sent_out = self.char_cnn(sent_char_emb_p)
            char_cnn_max_out[:, sent_i, :], _ = torch.max(char_cnn_sent_out, dim=2)
        char_cnn = self.cnn_dropout(char_cnn_max_out)
        # concat word and char embedding
        # char_cnn_p = [sentence length, batch size, char emb dim * num filter]
        char_cnn_p = char_cnn.permute(1, 0, 2)
        word_features = torch.cat((embedding_out, char_cnn_p), dim=2)
        # lstm_out = [sentence length, batch size, hidden dim * 2]
        lstm_out, _ = self.lstm(word_features)
        ### BEGIN MODIFIED SECTION: ATTENTION ###
        # create masking for paddings
        # key_padding_mask = [batch size, sentence length]
        key_padding_mask = torch.as_tensor(words == self.word_pad_idx).permute(1, 0)
        # attn_out = [sentence length, batch size, hidden dim * 2]
        # attn_weight = [batch size, sentence length, sentence length]
        attn_out, attn_weight = self.attn(lstm_out, lstm_out, lstm_out, key_padding_mask=key_padding_mask)
        # fc_out = [sentence length, batch size, output dim]
        fc_out = self.fc(self.fc_dropout(attn_out))
        crf_mask = words != self.word_pad_idx
        crf_out = self.crf.decode(fc_out, mask=crf_mask)
        crf_loss = -self.crf(fc_out, tags=tags, mask=crf_mask) if tags is not None else None
        return crf_out, crf_loss, attn_weight
        ### END MODIFIED SECTION: ATTENTION ###

    def init_embeddings(self, char_pad_idx, word_pad_idx, pretrained=None, freeze=True):
        # initialize embedding for padding as zero
        self.embedding.weight.data[word_pad_idx] = torch.zeros(self.embedding_dim)
        self.char_emb.weight.data[char_pad_idx] = torch.zeros(self.char_emb_dim)
        if pretrained is not None:
            self.embedding = nn.Embedding.from_pretrained(
                embeddings=torch.as_tensor(pretrained),
                padding_idx=word_pad_idx,
                freeze=freeze
            )


In [235]:
len(corpus.char_field.vocab)

146

In [237]:
bilstm = BiLSTM(
    input_dim=len(corpus.word_field.vocab),
    embedding_dim=300,
    char_emb_dim=25,
    char_input_dim=len(corpus.char_field.vocab),
    char_cnn_filter_num=5,
    char_cnn_kernel_size=3,
    hidden_dim=64,
    output_dim=len(corpus.tag_field.vocab),
    lstm_layers=2,
    attn_heads=16,
    emb_dropout=0.5,
    cnn_dropout=0.25,
    lstm_dropout=0.1,
    attn_dropout=0.25,
    fc_dropout=0.25,
    word_pad_idx=corpus.word_pad_idx,
    char_pad_idx=corpus.char_pad_idx,
    tag_pad_idx=corpus.tag_pad_idx
)
bilstm.init_embeddings(
    char_pad_idx=corpus.char_pad_idx,
    word_pad_idx=corpus.word_pad_idx,
    pretrained = None,
    freeze=True
)


print(bilstm)

The model has 2,174,184 trainable parameters.
BiLSTM(
  (embedding): Embedding(5841, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (char_emb): Embedding(146, 25, padding_idx=1)
  (char_cnn): Conv1d(25, 125, kernel_size=(3,), stride=(1,), groups=25)
  (cnn_dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(425, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
  )
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=7, bias=True)
  (crf): CRF(num_tags=7)
)


In [375]:
class Trainer(object):

    def __init__(self, model, data, optimizer_cls, loss_fn_cls):
        self.model = model
        self.data = data
        self.optimizer = optimizer_cls(model.parameters())
        self.loss_fn = loss_fn_cls(ignore_index=self.data.tag_pad_idx)

    @staticmethod
    def epoch_time(start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def accuracy(self, preds, y):
        flatten_preds = [pred for sent_pred in preds for pred in sent_pred]
        flatten_y = [tag for sent_tag in y for tag in sent_tag]
        correct = [pred == tag for pred, tag in zip(flatten_preds, flatten_y)]
        return sum(correct) / len(correct) if len(correct) > 0 else 0

    def epoch(self):
        epoch_loss = 0
        epoch_acc = 0
        self.model.train()
        for batch in self.data.train_iter:
            # words = [sent len, batch size]
            words = batch.word
            # chars = [batch size, sent len, char len]
            chars = batch.char
            # tags = [sent len, batch size]
            true_tags = batch.tag
            self.optimizer.zero_grad()
            pred_tags_list, batch_loss, _ = self.model(words, chars, true_tags)
            true_tags_list = [
                [tag for tag in sent_tag if tag != self.data.tag_pad_idx]
                for sent_tag in true_tags.permute(1, 0).tolist()
            ]
            batch_acc = self.accuracy(pred_tags_list, true_tags_list)
            batch_loss.backward()
            self.optimizer.step()
            epoch_loss += batch_loss.item()
            epoch_acc += batch_acc
        return epoch_loss / len(self.data.train_iter), epoch_acc / len(self.data.train_iter)

    def evaluate(self, iterator):
        epoch_loss = 0
        epoch_acc = 0
        self.model.eval()
        with torch.no_grad():
            for batch in iterator:
                words = batch.word
                chars = batch.char
                true_tags = batch.tag
                pred_tags, batch_loss, _ = self.model(words, chars, true_tags)
                true_tags_list = [
                    [tag for tag in sent_tag if tag != self.data.tag_pad_idx]
                    for sent_tag in true_tags.permute(1, 0).tolist()
                ]
                batch_acc = self.accuracy(pred_tags, true_tags_list)
                epoch_loss += batch_loss.item()
                epoch_acc += batch_acc
        return epoch_loss / len(iterator), epoch_acc / len(iterator)

    def train(self, n_epochs):
        for epoch in range(n_epochs):
            start_time = time.time()
            train_loss, train_acc = self.epoch()
            end_time = time.time()
            epoch_mins, epoch_secs = Trainer.epoch_time(start_time, end_time)
            print(f"Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
            print(f"\tTrn Loss: {train_loss:.3f} | Trn Acc: {train_acc * 100:.2f}%")
            val_loss, val_acc = self.evaluate(self.data.val_iter)
            print(f"\tVal Loss: {val_loss:.3f} | Val Acc: {val_acc * 100:.2f}%")
        test_loss, test_acc = self.evaluate(self.data.test_iter)
        print(f"Test Loss: {test_loss:.3f} |  Test Acc: {test_acc * 100:.2f}%")

    def infer(self, sentence, true_tags=None):
        self.model.eval()
        # tokenize sentence
        tokens = [token.text for token in my_tokenize(sentence)]
        max_word_len = max([len(token) for token in tokens])
        max_word_len = max(3, max_word_len)
        # transform to indices based on corpus vocab
        numericalized_tokens = [self.data.word_field.vocab.stoi[token.lower()] for token in tokens]
        numericalized_chars = []
        char_pad_id = self.data.char_pad_idx
        for token in tokens:
            numericalized_chars.append(
                [self.data.char_field.vocab.stoi[char] for char in token]
                + [char_pad_id for _ in range(max_word_len - len(token))]
            )
        # find unknown words
        unk_idx = self.data.word_field.vocab.stoi[self.data.word_field.unk_token]
        unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
        # begin prediction
        token_tensor = torch.as_tensor(numericalized_tokens)
        token_tensor = token_tensor.unsqueeze(-1)
        char_tensor = torch.as_tensor(numericalized_chars)
        char_tensor = char_tensor.unsqueeze(0)
        predictions, _, attn_weight = self.model(token_tensor, char_tensor)  # NEWLY ADDED
        #Trainer.visualize_attn(tokens, attn_weight.detach().numpy()[0])  # NEWLY ADDED
        # convert results to tags
        predicted_tags = [self.data.tag_field.vocab.itos[t] for t in predictions[0]]
        # print inferred tags
        max_len_token = max([len(token) for token in tokens] + [len('word')])
        max_len_tag = max([len(tag) for tag in predicted_tags] + [len('pred')])
#         print(
#             f"{'word'.ljust(max_len_token)}\t{'unk'.ljust(max_len_token)}\t{'pred tag'.ljust(max_len_tag)}"
#             + ("\ttrue tag" if true_tags else "")
#         )
#         for i, token in enumerate(tokens):
#             is_unk = "✓" if token in unks else ""
#             print(
#                 f"{token.ljust(max_len_token)}\t{is_unk.ljust(max_len_token)}\t{predicted_tags[i].ljust(max_len_tag)}"
#                 + (f"\t{true_tags[i]}" if true_tags else "")
#             )
        return tokens, predicted_tags, unks

In [243]:
trainer = Trainer(
    model=bilstm,
    data=corpus,
    optimizer_cls=Adam,
    loss_fn_cls=nn.CrossEntropyLoss  # this is no longer in use, keeping as-is for simplicity
)
trainer.train(50)

Epoch: 01 | Epoch Time: 0m 21s
	Trn Loss: 0.069 | Trn Acc: 99.98%
	Val Loss: 2.994 | Val Acc: 99.72%
Epoch: 02 | Epoch Time: 0m 21s
	Trn Loss: 0.010 | Trn Acc: 99.99%
	Val Loss: 2.470 | Val Acc: 99.81%
Epoch: 03 | Epoch Time: 0m 21s
	Trn Loss: 0.026 | Trn Acc: 100.00%
	Val Loss: 2.030 | Val Acc: 99.79%
Epoch: 04 | Epoch Time: 0m 21s
	Trn Loss: 0.170 | Trn Acc: 99.96%
	Val Loss: 1.738 | Val Acc: 99.80%
Epoch: 05 | Epoch Time: 0m 21s
	Trn Loss: 0.019 | Trn Acc: 99.99%
	Val Loss: 2.183 | Val Acc: 99.74%
Epoch: 06 | Epoch Time: 0m 20s
	Trn Loss: 0.007 | Trn Acc: 99.99%
	Val Loss: 2.122 | Val Acc: 99.78%
Epoch: 07 | Epoch Time: 0m 20s
	Trn Loss: 0.005 | Trn Acc: 100.00%
	Val Loss: 2.565 | Val Acc: 99.77%
Epoch: 08 | Epoch Time: 0m 21s
	Trn Loss: 0.052 | Trn Acc: 99.98%
	Val Loss: 2.321 | Val Acc: 99.75%
Epoch: 09 | Epoch Time: 0m 20s
	Trn Loss: 0.324 | Trn Acc: 99.93%
	Val Loss: 2.083 | Val Acc: 99.74%
Epoch: 10 | Epoch Time: 0m 20s
	Trn Loss: 0.032 | Trn Acc: 99.99%
	Val Loss: 1.961 | Val 

In [244]:
sentences = [test[index].text for index in range(len(test))]
labels = [test[index].labels for index in range(len(test))]

In [245]:
total_words = []
total_infer_tags = []
total_true_tags = []
for i, sentence in enumerate(sentences):
    tags = [label.name for label in labels[i]]
    words, infer_tags, unknown_tokens = trainer.infer(sentence=sentence, true_tags=tags)
    total_words.append(words)
    total_infer_tags.append(torch.tensor([ Label.__members__[tag].value for tag in infer_tags]))
    total_true_tags.append(torch.tensor([ Label.__members__[tag].value for tag in tags]))

In [246]:
def get_spans(labels, tokens):
    spans = []
    for i, (label, token) in enumerate(zip(labels, tokens)):
        if i != 0 and label == old_label:
          spans[-1]= (old_start, token.stop, old_tag)
          continue
        if label == 1:
          spans.append((token.start, token.stop, "ms_major_amount"))
          tag = "ms_major_amount"
        elif label == 2:
          spans.append((token.start, token.stop, "ms_major_currency"))
          tag = "ms_major_currency"
        elif label == 3:
          spans.append((token.start, token.stop, "ms_minor_amount"))
          tag = "ms_minor_amount"
        elif label == 4:
          spans.append((token.start, token.stop, "ms_minor_currency"))
          tag = "ms_minor_currency"
        elif label == 5:
          spans.append((token.start, token.stop, "ms_major_amount_bracketed"))
          tag = "ms_major_amount_bracketed"
        elif label == 6:
          spans.append((token.start, token.stop, "like_brackets"))
          tag = "like_brackets"
        elif label == 0:
          spans.append((token.start, token.stop, "outer"))
          tag = "outer"
        old_label = label
        old_start = token.start
        old_tag = tag
    return spans


def calc_metrics(true_labels, predicted_labels, tokens):
    # Метрики классификации
    all_true_labels = torch.tensor((len(test), 27))
    all_predicted_labels = torch.tensor((len(test), 27))
    torch.cat(true_labels, out=all_true_labels)
    torch.cat(predicted_labels, out=all_predicted_labels)
    print(all_true_labels, all_predicted_labels)
    confusion = confusion_matrix(all_true_labels, all_predicted_labels)
    print('Confusion Matrix\n')
    print(confusion)
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(all_true_labels, all_predicted_labels)))

    print('Micro Precision: {:.2f}'.format(precision_score(all_true_labels, all_predicted_labels, average='micro')))
    print('Micro Recall: {:.2f}'.format(recall_score(all_true_labels, all_predicted_labels, average='micro')))
    print('Micro F1-score: {:.2f}\n'.format(f1_score(all_true_labels, all_predicted_labels, average='micro')))

    print('Macro Precision: {:.2f}'.format(precision_score(all_true_labels, all_predicted_labels, average='macro')))
    print('Macro Recall: {:.2f}'.format(recall_score(all_true_labels, all_predicted_labels, average='macro')))
    print('Macro F1-score: {:.2f}\n'.format(f1_score(all_true_labels, all_predicted_labels, average='macro')))

    print('Weighted Precision: {:.2f}'.format(precision_score(all_true_labels, all_predicted_labels, average='weighted')))
    print('Weighted Recall: {:.2f}'.format(recall_score(all_true_labels, all_predicted_labels, average='weighted')))
    print('Weighted F1-score: {:.2f}'.format(f1_score(all_true_labels, all_predicted_labels, average='weighted')))

    from sklearn.metrics import classification_report
    print('\nClassification Report\n')
    print(classification_report(all_true_labels, all_predicted_labels, target_names=['ms_major_amount', 'ms_major_currency', 'ms_minor_amount', 'ms_minor_currency', 'ms_major_amount_bracketed', 'outer_like_brackets']))

    

calc_metrics(total_true_tags, total_infer_tags, total_words)

tensor([1, 2, 3,  ..., 2, 2, 1]) tensor([1, 2, 3,  ..., 2, 2, 1])
Confusion Matrix

[[5392    5    1    0    0    6]
 [   3 5048    0    0    0    5]
 [   2    0   20    0    0    0]
 [   0    2    0   13    0    0]
 [   0    0    0    0   15    0]
 [   1    1    0    0    0   41]]

Accuracy: 1.00

Micro Precision: 1.00
Micro Recall: 1.00
Micro F1-score: 1.00

Macro Precision: 0.96
Macro Recall: 0.95
Macro F1-score: 0.95

Weighted Precision: 1.00
Weighted Recall: 1.00
Weighted F1-score: 1.00

Classification Report

                           precision    recall  f1-score   support

          ms_major_amount       1.00      1.00      1.00      5404
        ms_major_currency       1.00      1.00      1.00      5056
          ms_minor_amount       0.95      0.91      0.93        22
        ms_minor_currency       1.00      0.87      0.93        15
ms_major_amount_bracketed       1.00      1.00      1.00        15
      outer_like_brackets       0.79      0.95      0.86        43

        

/tmp/ipykernel_111/4198949536.py:38: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [10555]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(true_labels, out=all_true_labels)
/tmp/ipykernel_111/4198949536.py:39: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [10555]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(predicte

In [247]:
def generate_false_data(all_true_labels, all_predicted_labels, all_tokens, all_texts):
    major_amount_synthetic, major_currency_synthetic, minor_amount_synthetic, minor_currency_synthetic, ms_major_amount_bracketed_sintetic, like_brackets_sintetic, outer_sintetic = [],[],[],[],[],[],[]
    cnt = 0
    for num in range(len(test)):
        eq = torch.all(torch.eq(all_true_labels[num], all_predicted_labels[num]) == True)
        if not eq:
            cnt +=1
            show_box_markup(all_texts[num],
                    get_spans(all_predicted_labels[num], all_tokens[num]),
                    palette=palette(ms_major_currency=BLUE, ms_major_amount=RED, ms_minor_amount=GREEN, ms_minor_currency=PURPLE, ms_major_amount_bracketed=BROWN))
            show_box_markup(all_texts[num],
                    get_spans(all_true_labels[num], all_tokens[num]),
                    palette=palette(ms_major_currency=BLUE, ms_major_amount=RED, ms_minor_amount=GREEN, ms_minor_currency=PURPLE, ms_major_amount_bracketed=BROWN))
            for i, (tr_l, pr_l) in enumerate(zip(all_true_labels[num], all_predicted_labels[num])):
                if tr_l != pr_l:
                    all_tokens
            print()
    return cnt

In [248]:
all_tokens = [my_tokenize(test[num].text) for num in range(len(test))]

In [249]:
all_texts = [test[num].text for num in range(len(test))]

In [250]:
false_detection = generate_false_data(total_true_tags, total_infer_tags, all_tokens, all_texts)

In [251]:
false_detection

18

## Add synthetics

In [255]:
!sudo pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=9da196e67ad5558b50c986ea21875e78dc9d7bcb7463b656315df71439bb1a6b
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [418]:
ms_major_currency_syntetics = ['1988 USD', 'Malayan dollars', 'BAHT', '.999-silver', '1988 USD', 'GH¢', '']
ms_major_amount_syntetics = ['thousand', 'MM', 'min', 'Mio', 'million', '']
ms_like_brackets_syntetics = ['(', 'and', '']
ms_minor_currency_syntetics = ['kop.', 'quadrans', 's', 'Cents', '']

In [437]:
import random
from num2words import num2words

def generate_synthetic_data(ms_major_amount_syntetics, ms_major_currency_syntetics, ms_like_brackets_syntetics, ms_minor_currency_syntetics):
  # select ms_major_amount
  number = random.randint(1, 100)
  number_text = num2words(number).replace('-', ' ')
  second_part = random.choice(ms_major_amount_syntetics)
  if random.uniform(0, 1) > 0.5:
    major_amount = str(number) + second_part
  else:
    major_amount = number_text + ' ' + second_part
  
  # select ms_major_currency
  major_currency = random.choice(ms_major_currency_syntetics)

  # select brackets
  outer_like_brackets = random.choice(ms_like_brackets_syntetics)
  if outer_like_brackets == '(':
    number = random.randint(11, 100)
    number_text = num2words(number).replace('-', ' ')
    outer_like_brackets += random.choice([number_text, random.choice(ms_major_amount_syntetics[:-1])]) + ')'
  elif outer_like_brackets == '[':
    number = random.randint(11, 100)
    number_text = num2words(number).replace('-', ' ')
    outer_like_brackets += random.choice([number_text, random.choice(ms_major_amount_syntetics[:-1])]) + ']'
  # select minors
  minor_currency = random.choice(ms_minor_currency_syntetics)
  if minor_currency == '':
    minor_amount = ''
  else:
    number = random.randint(1, 50)
    minor_amount = num2words(number).replace('-', ' ')

  anotation_str = ""
  anotation_list = []
  prefix = 0
  if major_amount != '':
    anotation_str += major_amount
    anotation_list.append((0, len(major_amount), 'ms_major_amount'))
    prefix = len(major_amount)
  if major_currency != '':
    anotation_str += ' ' + major_currency
    anotation_list.append((prefix + 1, prefix + len(major_currency) + 1, 'ms_major_currency'))
    prefix += len(major_currency) + 1

  if outer_like_brackets != '':
    anotation_str += ' ' + outer_like_brackets
    if outer_like_brackets[0] == '(':
      anotation_list.append((prefix+2, prefix + len(outer_like_brackets), 'ms_major_amount_bracketed'))
    elif outer_like_brackets[0] == '[':
      anotation_list.append((prefix+2, prefix + len(outer_like_brackets), 'ms_major_amount_bracketed'))
    prefix += len(outer_like_brackets) + 1
  
  if minor_amount != '':
    anotation_str += ' ' + minor_amount
    anotation_list.append((prefix + 1, prefix + 1 + len(minor_amount), 'ms_minor_amount'))
    prefix += 1 + len(minor_amount)
  
  if minor_currency != '':
    anotation_str += ' ' + minor_currency
    anotation_list.append((prefix + 1, prefix + 1 + len(minor_currency), 'ms_minor_currency'))
    prefix += 1 + len(minor_currency)

  return (anotation_str, anotation_list)

synthetic_data = []
for _ in range(120):
  synthetic_data.append(generate_synthetic_data(ms_major_amount_syntetics, ms_major_currency_syntetics, ms_like_brackets_syntetics, ms_minor_currency_syntetics))

show_box_markup(synthetic_data[2][0], synthetic_data[2][1], palette=palette(ms_major_currency=BLUE, ms_major_amount=RED, ms_minor_amount=GREEN, ms_minor_currency=PURPLE, ms_major_amount_bracketed=BROWN))

In [438]:
synthetic_train = []
for text, spans in synthetic_data:
    synthetic_train.append(text_span_to_sample(text, spans))

In [439]:
new_synthetic_samples = samples + synthetic_train
new_synthetic_train = train + synthetic_train

In [440]:
with open('input/synthetic_train.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for index in range(len(new_synthetic_train)):
        for token_index in range(len(new_synthetic_train[index].tokens)):
            tsv_writer.writerow([new_synthetic_train[index].tokens[token_index].text, new_synthetic_train[index].labels[token_index].name])
        tsv_writer.writerow(['', ''])

with open('input/synthetic_test.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for index in range(len(test)):
        for token_index in range(len(test[index].tokens)):
            tsv_writer.writerow([test[index].tokens[token_index].text, test[index].labels[token_index].name])
        tsv_writer.writerow(['', ''])

with open('input/synthetic_val.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for index in range(len(val)):
        for token_index in range(len(val[index].tokens)):
            tsv_writer.writerow([val[index].tokens[token_index].text, val[index].labels[token_index].name])
        tsv_writer.writerow(['', ''])

In [441]:
class Corpus(object):

    def __init__(self, input_folder, min_word_freq, batch_size, wv_file=None):
        # list all the fields
        self.word_field = Field(lower=True)  # [sent len, batch_size]
        self.tag_field = Field(unk_token=None)  # [sent len, batch_size]
        # Character-level input
        self.char_nesting_field = Field(tokenize=list)
        self.char_field = NestedField(self.char_nesting_field)  # [batch_size, sent len, max len char]
        # create dataset using built-in parser from torchtext
        self.train_dataset, self.val_dataset, self.test_dataset = SequenceTaggingDataset.splits(
            path=input_folder,
            train="synthetic_train.tsv",
            validation="synthetic_val.tsv",
            test="synthetic_test.tsv",
            fields=(
                (("word", "char"), (self.word_field, self.char_field)),
                ("tag", self.tag_field)
            )
        )
    
        # convert fields to vocabulary list
        self.word_field.build_vocab(self.train_dataset.word, min_freq=min_word_freq)
        # build vocab for tag and characters
        self.char_field.build_vocab(self.train_dataset.char)
        self.tag_field.build_vocab(self.train_dataset.tag)
        # create iterator for batch input
        self.train_iter, self.val_iter, self.test_iter = BucketIterator.splits(
            datasets=(self.train_dataset, self.val_dataset, self.test_dataset),
            batch_size=batch_size
        )
        # prepare padding index to be ignored during model training/evaluation
        self.word_pad_idx = self.word_field.vocab.stoi[self.word_field.pad_token]
        self.char_pad_idx = self.char_field.vocab.stoi[self.char_field.pad_token]
        self.tag_pad_idx = self.tag_field.vocab.stoi[self.tag_field.pad_token]

In [442]:
synthetic_corpus = Corpus('/home/devel/NVI/outputs/my_notebook/input/', 1, batch_size = 16)

In [443]:
bilstm = BiLSTM(
    input_dim=len(synthetic_corpus.word_field.vocab),
    embedding_dim=300,
    char_emb_dim=25,
    char_input_dim=len(synthetic_corpus.char_field.vocab),
    char_cnn_filter_num=5,
    char_cnn_kernel_size=3,
    hidden_dim=64,
    output_dim=len(synthetic_corpus.tag_field.vocab),
    lstm_layers=2,
    attn_heads=16,
    emb_dropout=0.5,
    cnn_dropout=0.25,
    lstm_dropout=0.1,
    attn_dropout=0.25,
    fc_dropout=0.25,
    word_pad_idx=synthetic_corpus.word_pad_idx,
    char_pad_idx=synthetic_corpus.char_pad_idx,
    tag_pad_idx=synthetic_corpus.tag_pad_idx
)
bilstm.init_embeddings(
    char_pad_idx=synthetic_corpus.char_pad_idx,
    word_pad_idx=synthetic_corpus.word_pad_idx,
    pretrained = None,
    freeze=True
)

print(f"The model has {bilstm.count_parameters():,} trainable parameters.")
print(bilstm)

The model has 2,176,284 trainable parameters.
BiLSTM(
  (embedding): Embedding(5848, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (char_emb): Embedding(146, 25, padding_idx=1)
  (char_cnn): Conv1d(25, 125, kernel_size=(3,), stride=(1,), groups=25)
  (cnn_dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(425, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
  )
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=7, bias=True)
  (crf): CRF(num_tags=7)
)


In [ ]:
trainer = Trainer(
    model=bilstm,
    data=synthetic_corpus,
    optimizer_cls=Adam,
    loss_fn_cls=nn.CrossEntropyLoss  # this is no longer in use, keeping as-is for simplicity
)
trainer.train(50)

Epoch: 01 | Epoch Time: 0m 23s
	Trn Loss: 20.715 | Trn Acc: 77.80%
	Val Loss: 3.076 | Val Acc: 98.61%
Epoch: 02 | Epoch Time: 0m 23s
	Trn Loss: 2.702 | Trn Acc: 98.37%
	Val Loss: 1.243 | Val Acc: 99.60%
Epoch: 03 | Epoch Time: 0m 23s
	Trn Loss: 1.395 | Trn Acc: 99.20%
	Val Loss: 1.130 | Val Acc: 99.58%
Epoch: 04 | Epoch Time: 0m 22s
	Trn Loss: 0.709 | Trn Acc: 99.58%
	Val Loss: 0.801 | Val Acc: 99.80%
Epoch: 05 | Epoch Time: 0m 22s
	Trn Loss: 0.437 | Trn Acc: 99.77%
	Val Loss: 0.799 | Val Acc: 99.73%
Epoch: 06 | Epoch Time: 0m 22s
	Trn Loss: 0.453 | Trn Acc: 99.76%
	Val Loss: 0.731 | Val Acc: 99.78%
Epoch: 07 | Epoch Time: 0m 23s
	Trn Loss: 0.234 | Trn Acc: 99.86%
	Val Loss: 1.058 | Val Acc: 99.75%
Epoch: 08 | Epoch Time: 0m 23s
	Trn Loss: 0.365 | Trn Acc: 99.79%
	Val Loss: 0.897 | Val Acc: 99.72%
Epoch: 09 | Epoch Time: 0m 25s
	Trn Loss: 0.195 | Trn Acc: 99.89%
	Val Loss: 0.794 | Val Acc: 99.84%


In [ ]:
sentences = [test[index].text for index in range(len(test))]
labels = [test[index].labels for index in range(len(test))]

In [ ]:
total_words = []
total_infer_tags = []
total_true_tags = []
for i, sentence in enumerate(sentences):
    tags = [label.name for label in labels[i]]
    words, infer_tags, unknown_tokens = trainer.infer(sentence=sentence, true_tags=tags)
    total_words.append(words)
    total_infer_tags.append(torch.tensor([ Label.__members__[tag].value for tag in infer_tags]))
    total_true_tags.append(torch.tensor([ Label.__members__[tag].value for tag in tags]))

In [ ]:
calc_metrics(total_true_tags, total_infer_tags, total_words)

In [ ]:
false_detection = generate_false_data(total_true_tags, total_infer_tags, all_tokens, all_texts)

In [ ]:
false_detection